# Imports

In [4]:
# from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import plotly.graph_objects as go

ModuleNotFoundError: No module named 'plotly'

# Tutorial on Beautiful Soup

In [72]:
page=7
URL = f"https://www.property24.com/to-rent/western-cape/9/p{page}?PropertyCategory=House%2cApartmentOrFlat"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "lxml")

In [76]:
properties = soup.find_all('div', class_='js_resultTile')
id = 12
print(properties[id]['data-listing-number'])
print(f"https://www.property24.com{properties[id].find('div', class_='js_rollover_container').a['href']}")

110855469
https://www.property24.com/to-rent/plattekloof-3/parow/western-cape/31186/110855469?plId=966055&plt=4


# Property24 Scraper

In [77]:
columns = ['id', 'link', 'title', 'description', 'bedrooms', 'bathrooms', 'parking_spaces', 'floor_area', 'location', 'address', 'rental_term', 'price']
df = pd.DataFrame([])
page = 0

while True:
    try:
        page += 1

        # URL = f"https://www.property24.com/apartments-to-rent/western-cape/9/p{page}"
        URL = f"https://www.property24.com/to-rent/western-cape/9/p{page}?PropertyCategory=House%2cApartmentOrFlat"
        soup = BeautifulSoup(requests.get(URL).content, "lxml")

        try:
            soup.find('div', {'class': 'panel-body text-danger text-center'}).h4.text
            break
        except:
            properties = soup.find_all('div', class_='js_resultTile')

        for property in properties:
            id = property['data-listing-number']
            link = f"https://www.property24.com{property.find('div', class_='js_rollover_container').a['href']}"
            
            
            try:
                price = property.find('span', class_='p24_price')['content'].strip()
            except:
                try:
                    price = property.find('div', class_='p24_price').text.strip().strip('R').replace(' ', '')
                except:
                    price = 'na'

            try:
                title = property.find('span', class_='p24_title').text.strip()
            except:
                try:
                    title = property.find('div', class_='p24_description').text.strip()
                except:
                    title = 'na'

            try:
                location = property.find('span', class_='p24_location').text.strip()
            except:
                try:
                    location = property.find('div', class_='p24_location').text.strip()
                except:
                    location = 'na'

            try:
                description = property.find('span', class_='p24_excerpt').text.strip()
            except:
                try:
                    description = property.find('span', class_='p24_excerpt').text.strip()
                except:
                    description = 'na'

            try:
                property.find('span', class_='left rentalTerm p24_rentalTerm').text.strip()
                rental_term = 'daily'
            except:
                rental_term = 'monthly'

            try:
                address = property.find('span', class_='p24_address').text.strip()
            except:
                address = 'na'

            try:
                bedrooms = property.find('span', {"class": "p24_featureDetails", 'title':'Bedrooms'}).text.strip()
            except:
                bedrooms = 'na'

            try:
                bathrooms = property.find('span', {"class": "p24_featureDetails", 'title':'Bathrooms'}).text.strip()
            except:
                bathrooms = 'na'

            try:
                parking_spaces = property.find('span', {"class": "p24_featureDetails", 'title':'Parking Spaces'}).text.strip()
            except:
                parking_spaces = 'na'

            try:
                floor_size = property.find('span', {"class": "p24_size", 'title':'Floor Size'}).text.strip().rstrip('m²').strip().replace(' ', '')
            except:
                floor_size = 'na'

            temp = pd.DataFrame([[id, link, title, description, bedrooms, bathrooms, parking_spaces, floor_size, location, address, rental_term, price]], columns=columns)

            df = pd.concat([df, temp], ignore_index=True)

            del temp
    
    except Exception as ex:
        print(ex)
        print(id)
        print("probably last page:", page)
        break # exit `while` loop

df.to_feather('western_cape_rentals.ftr')

In [2]:
df = pd.read_feather('western_cape_rentals_processed.ftr')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6569 entries, 0 to 6568
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              6569 non-null   int64  
 1   link            6569 non-null   object 
 2   title           6569 non-null   object 
 3   description     6569 non-null   object 
 4   bedrooms        6257 non-null   float64
 5   bathrooms       6331 non-null   float64
 6   parking_spaces  5008 non-null   float64
 7   floor_area      2266 non-null   float64
 8   location        6569 non-null   object 
 9   address         2105 non-null   object 
 10  rental_term     6569 non-null   object 
 11  price           6470 non-null   float64
dtypes: float64(5), int64(1), object(6)
memory usage: 616.0+ KB


In [79]:
df = df.replace('na', np.nan).copy()

In [4]:
df.sample(50)

,id,link,title,description,bedrooms,bathrooms,parking_spaces,floor_area,location,address,rental_term,price
426,110983258,https://www.property24.com/to-rent/paarl-centr...,1 Bedroom House,Main bedroom with a loose standing wardrobe.\n...,1.0,1.0,1.0,NaN,Paarl Central,None,monthly,7800.0
6392,107511943,https://www.property24.com/to-rent/hermanus-ce...,2 Bedroom Apartment,SEASONAL DISCOUNTS:\r\nPeak Season: Reduced fo...,2.0,2.0,1.0,NaN,Hermanus Central,None,daily,5000.0
5121,110546266,https://www.property24.com/to-rent/klein-neder...,2 Bedroom Apartment,This two Bedroom Apartment is situated in Vine...,2.0,2.0,NaN,NaN,Klein Nederburg,None,monthly,7850.0
5202,110713771,https://www.property24.com/to-rent/keurbooms-r...,4 Bedroom House,4 bedrooms/4 bathrooms - sleeps 8.\nSet in the...,4.0,4.0,4.0,NaN,Keurbooms River,None,daily,21800.0
6472,104194100,https://www.property24.com/to-rent/llandudno/c...,5 Bedroom House,This beautiful 5 bedroom luxury villa with pan...,5.0,4.0,2.0,NaN,Llandudno,None,monthly,70000.0
6289,110835340,https://www.property24.com/to-rent/brooklyn/mi...,1 Bedroom Apartment,We are pleased to offer this neat 1 Bedroom Ap...,1.0,1.0,NaN,NaN,Brooklyn,None,monthly,4500.0
6217,110544902,https://www.property24.com/to-rent/clifton/cap...,2 Bedroom Apartment,This two-bedroom apartment is situated on Vict...,2.0,2.0,1.0,NaN,Clifton,None,daily,NaN
6286,109137410,https://www.property24.com/to-rent/sea-point/c...,2 Bedroom Apartment,This semi retirement living at the Horizons Ap...,2.0,1.0,1.0,NaN,Sea Point,None,monthly,16000.0
2944,110919948,https://www.property24.com/to-rent/parow-north...,1 Bedroom House,This beautiful 1 bedroom flat is situated in a...,1.0,1.0,1.0,NaN,Parow North,None,monthly,7500.0
6013,110788281,https://www.property24.com/to-rent/loevenstein...,2 Bedroom Apartment,Semi-open plan kitchen to lounge. Upstairs you...,2.0,1.0,1.0,NaN,Loevenstein,None,monthly,6900.0


In [81]:
df.id = df.id.astype('int64')
df.bedrooms = df.bedrooms.astype('float64')
df.bathrooms = df.bathrooms.astype('float64')
df.parking_spaces = df.parking_spaces.astype('float64')
df.floor_area = df.floor_area.str.replace(' ', '')
df.floor_area = df.floor_area.astype('float64')
df.price = df.price.astype('float64')

In [82]:
df.to_feather('western_cape_rentals_processed.ftr')

In [83]:
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_file("Western Cape Rentals Processed.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 138.21it/s]


In [ ]:
western_cape = "https://www.property24.com/for-sale/all-cities/western-cape/9"
gauteng = "https://www.property24.com/for-sale/all-cities/gauteng/1"

In [ ]:
# Query Categories
ACTIONS = ["for-sale", "to-rent"]
PROPERTY_CATEGORIES = ["House", "ApartmentOrFlat", "Townhouse", "VacantLandOrPlot", "Farm", "Commercial", "Industrial"]
PARKING_TYPES = ["ParkingOrGarage", "GarageOnly"]

# Query Syntax
QUERY = "?"
OR = "%2c"
AND = "%26"
MIN_PRICE = "pf%3d"
MAX_PRICE = "pt%3d"
BEDROOMS = "bd%3d"
BATHROOM = "bth%3d"
PARKING_SPACES = "ps%3d"
PARKING_TYPE = "pst%3d"

# Query variables
PROPERTY_CATEGORY = "PropertyCategory="
FILTER = "sp="

In [2]:
URL = "https://www.property24.com/to-rent/somerset-west-central/somerset-west/western-cape/9019/110768093?plId=967801&plt=3"

page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [3]:
results = soup.find_all("div", class_="p24_listingContent")

In [4]:
for result in results:
    price = result.find("div", class_="p24_listingCard p24_listingFeaturesWrapper")
    # company_element = result.find("h3", class_="company")
    # location_element = result.find("p", class_="location")

    print(price.text.strip())

R 8 500
